In [1]:
import os, sys
import pickle
import pygrib
import datetime
import numpy as np
import numpy.ma as ma
import collections
%matplotlib inline
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from  more_itertools import unique_everseen
plt.rcParams['figure.figsize'] = 16, 12

## Reading 3d salinity from Hiromb BS01 file
Salinity are used as a land/sea mask.

In [17]:
lev = [grbMsg.level for grbMsg in filter(lambda grbMsg: grbMsg.name in ['Salinity'],
                                         pygrib.open('BS01_201611290000+000H00M'))]
sal = np.array([grbMsg.values for grbMsg in filter(lambda grbMsg: grbMsg.name in ['Salinity'],
                                                   pygrib.open('BS01_201611290000+000H00M'))])

In [18]:
lat = [round(c, 4) for c in unique_everseen(f[1]['latitudes'])]
lon = [round(c, 4) for c in unique_everseen(f[1]['longitudes'])]

## Reading 3d salinity from Nemo-Nordic NS01 file

In [21]:
f2 = Dataset('NS01_3D_2016112900.nc')

In [22]:
sal2 = np.array(f2.variables['salinity'][0,:,:,:])

In [23]:
lev2 = [round(d) for d in f2.variables['deptht'][:]]
lat2 = [round(c, 4) for c in unique_everseen(f2.variables['nav_lat'][:].ravel())]
lat2 = sorted(lat2)[1:]
lon2 = [round(c, 4) for c in unique_everseen(f2.variables['nav_lon'][:].ravel())]
lon2 = sorted(lon2)[1:]

## Function for nudging cell indexies into water

In [24]:
def search_wet(field, x, y, n):
    for nudge in ((0,1), (1,0), (0,-1), (-1,0), (1,2), (2,1), (1,-2),
                (-2,1), (-1,2), (2,-1), (2,2), (-2,2), (2,-2), (-2,-2)):
        try:
            if field[n, y+nudge[1], x+nudge[0]] < 9999:
                return (x+nudge[0], y+nudge[1], n)
        except:
            continue

## Looping over all cells in target grid. Finding the closest cell in the input grid.

In [26]:
amap = {}
for n in range(sal.shape[0]):
    n2 = min(range(len(lev2)), key=lambda i: abs(lev2[i]-lev[n]))
    for x in range(sal[0].shape[1]):
        for y in range(sal[0].shape[0]):
            if sal[n, y, x] != 9999:
                try:
                    x2 = lon2.index(lon[x])
                except:
                    x2 = min(range(len(lon2)), key=lambda i: abs(lon2[i]-lon[x]))
                try:
                    y2 = lat2.index(lat[y]-0.05)
                except:
                    y2 = min(range(len(lat2)), key=lambda i: abs(lat2[i]-(lat[y]-0.05)))
                if sal2[n2, y2, x2] == 1:
                    amap[(x, y, n)] = (x2, y2, n2)
                else:
                    amap[(x, y, n)] = search_wet(sal2, x2, y2, n2)

### Number of wet cells in BS01

In [27]:
len(amap)

1918522

### Save map, bs01 grid skelleton and bs01 levels in cPickle file

In [26]:
bs01_skel = sal

In [27]:
pickle.dump((amap, bs01_skel, lev), open('/home/a001988/Documents/bs01_map5.p', 'wb'))